### bilibili使用js动态加载，直接爬虫爬不到，需要在f12控制台报文中找到调用的api。

#### 爬取用户关注数据
以我为原点爬取我的关注和我的粉丝，再依次爬取他们的关注和粉丝
数据保存在userdf 和 followdf 中

### 200615 v1.1 重构代码，重写get为getjson。

In [1]:
import time
import math
import requests
import pandas as pd
headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
    'Referer':"https://www.bilibili.com/"
}
# cookies = {
#     "SESSDATA": "0ccb2e9a%2C1601886871%2C7146f*41"
# }


                
########## 工具类 

# 为dataFrame 添加新行
def addDf(df, data):
    df.loc[df.shape[0],:] = data
    
# 获取json信息。
def getJson(url):
    try:
        r = requests.get(url, headers=headers, timeout=5)
        if r.status_code == 412:
            print("当前时间：",time.localtime().tm_hour," : ",time.localtime().tm_min," 休眠一次")
            time.sleep(5*60)
            return {}
        else:
            return r.json()
    except:
        return {}

def saveData():
    userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",  encoding='utf_8_sig')
    followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",  encoding='utf_8_sig')
    
def roadData():
    userdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",index_col=0, dtype={'uid':int,'follower':int,'level':int,'coins':int,'following':int})
    followdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",index_col=0, dtype='int')
    return userdf, followdf

########## 具体类
        
#获取关注数fi 和粉丝数fe 。用于做列表爬虫。
def getFfNum(mid):
    api = "https://api.bilibili.com/x/relation/stat?vmid="+str(mid)+"&jsonp=jsonp"
    json = getJson(api)
    if 'data' in json:
        fi = json['data']['following']
        fe = json['data']['follower']
        return int(fi), int(fe)
    else:
        return -1, -1
    
def getUserName(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    json = getJson(api)
    if 'data' in json:
        return str(json['data']['name'])
    else:
        return 'null'


# 在userdf中添加数据
def addUser(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    json = getJson(api)
    fi, fe = getFfNum(mid)
    if 'data' in json and json['data'] and fi != -1 and fe != -1:
        data = json['data']
        addDf(userdf, [data['mid'], data['name'], data['sex'], data['level'], data['birthday'], data['coins'], fi, fe])

# 获取用户mid 关注列表filist
def getFollowingsList(mid):
    if mid not in userdf['uid'].values:
        addUser(mid)
    fi = userdf[userdf['uid']==mid]['following'].values
    if len(fi)<1:
        print('用户',str(mid),'失败')
        return []
    
    flist = []
    # 除20 向上取整
#     page = math.ceil(fi[0]/20)
    # biliapi 只能访问5页限制。
    page = 5
    for p in range(page):
        api = "https://api.bilibili.com/x/relation/followings?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
        json = getJson(api)
        if 'data' in json and json['data'] and 'list' in json['data']:
            for info in json['data']['list']:
                flist.append(int(info['mid']))
                addDf(followdf, [mid, info['mid'] ])
#     print('用户',str(mid),'完成')
    saveData()
    return flist

# 封装后的getfollow
def getFollowings(mid):
    if type(mid) == list:
        print(mid) # 
        flist = []
        isok = 0
        for m in mid:
            f = getFollowingsList(m)
            if not f:
                isok += 1
            flist.extend(f)
        print('搜取数据数量： ',isok,'  数据搜取成功率： ', isok/len(mid))
        return flist
    else :
        return getFollowingsList(mid)

    
def getFollowersList(mid):
    if mid not in userdf['uid'].values:
        addUser(mid)
    fe = userdf[userdf['uid']==mid]['following'].values
    if len(fe)<1:
        return []
    flist = []
#     除20 向上取整
#     page = math.ceil(fe[0]/20)
    # biliapi 只能访问5页限制。
    page = 5
    for p in range(page):
        api = "https://api.bilibili.com/x/relation/followers?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
        json = getJson(api)

        if 'data' in json and json['data'] and 'list' in json['data']:
            for info in json['data']['list']:
                flist.append(int(info['mid']))
    return flist

def getFollowers(mid):
    if type(mid) == list:
        print(mid) # 
        flist = []
        isok = 0
        for m in mid:
            f = getFollowersList(m)
            if not f:
                isok += 1
            flist.extend(f)
        print('搜取数据数量： ',isok,'  数据搜取成功率： ', isok/len(mid))
        return flist
    else :
        return getFollowersList(mid)


In [ ]:
# 获取某用户相关用户
userdf, followdf = roadData()
def main():
    
    my = 58289054
    # 获取用户的关注数据
    list1 = getFollowings(my)
#     list1 = [456989566, 254463269, 17546432, 3811052, 344194038, 430837803, 449825072, 382193067, 13677047, 35727555, 80717854, 927587, 20055735, 338927049, 21950148, 25782796, 407007380, 11061327, 28751839, 17075276, 6292102, 288251292, 6700597, 6302909, 18020792, 39627524, 326081112, 40966108, 23719433, 240346507, 883968, 135589565, 95304526, 16166225, 383665157, 215910135, 24127511]

    # 获取被关注的人的粉丝
    list2 = getFollowers(list1)
    
    # 去除重复
    list3 = list(set(list2) - set(list1))
    
    # 对粉丝再次获取关注数据
    list4 = getFollowings(list3)
    
    list5 = getFollowers(list4)
    
    list6 = list(set(list5) - set(list4))
    
    getFollowings(list6)
    
main()

[280793434, 416186469, 350001797, 257797153, 102043515, 526294618, 7552204, 178029850, 208259, 479893558, 506277449, 385128992, 384951376, 64840236, 380128550, 24952714, 92609386, 6926237, 427639872, 245895278, 5970160, 5294454, 30895245, 472586604, 517327498, 17223352, 615236, 346796186, 35461776, 493590366, 344509109, 343055119, 88461692, 403943112, 7788379, 7753700, 221648, 344194038, 21837784, 38136734, 217049242, 357451181, 121682346, 204086589, 268819811, 350905839, 437257786, 470647022, 362169745, 927587, 269380821, 386080586, 23640791, 415270918, 7411106, 66391032, 12413745, 40096793, 32847571, 178534406, 41292603, 382038930, 2884629, 16166225, 8188433, 275153040, 8303290, 52074480, 340223423, 370342489, 15967711, 2284412, 27511366, 234557264, 383289740, 103496542, 330325021, 175456549, 13607452, 12333557, 7720791, 105034258, 330881052, 87641747, 26013256, 313861]
搜取数据数量：  0   数据搜取成功率：  0.0
[444203012, 479199236, 622329864, 202113039, 196630, 555974679, 354254873, 290553883, 56

当前时间： 9  :  19  休眠一次
用户 470909150 失败
当前时间： 9  :  24  休眠一次
用户 404586719 失败
当前时间： 9  :  29  休眠一次
用户 425459939 失败
当前时间： 9  :  34  休眠一次
用户 383320297 失败
当前时间： 9  :  39  休眠一次
用户 88441070 失败
当前时间： 9  :  44  休眠一次
用户 294879472 失败
当前时间： 9  :  57  休眠一次
当前时间： 10  :  2  休眠一次
当前时间： 10  :  7  休眠一次
用户 622330657 失败
当前时间： 10  :  12  休眠一次
当前时间： 10  :  17  休眠一次
用户 488178465 失败
当前时间： 10  :  22  休眠一次
用户 278004515 失败
当前时间： 10  :  27  休眠一次
用户 287376170 失败
当前时间： 10  :  41  休眠一次
当前时间： 10  :  46  休眠一次
用户 620201411 失败
当前时间： 10  :  51  休眠一次
用户 509511108 失败
当前时间： 10  :  56  休眠一次
用户 587236807 失败
当前时间： 11  :  1  休眠一次
用户 21169612 失败
当前时间： 11  :  6  休眠一次
用户 501745101 失败
当前时间： 11  :  19  休眠一次
用户 440961051 失败
当前时间： 11  :  24  休眠一次
用户 430311452 失败
当前时间： 11  :  29  休眠一次
用户 399902759 失败
当前时间： 11  :  34  休眠一次
用户 392759341 失败
当前时间： 11  :  39  休眠一次
用户 320571443 失败
当前时间： 11  :  44  休眠一次
用户 35981369 失败
当前时间： 11  :  57  休眠一次
用户 609880733 失败
当前时间： 12  :  2  休眠一次
用户 854686 失败
当前时间： 12  :  7  休眠一次
用户 448236197 失败
当前时间： 12  :  12  休眠

### v1.0

In [3]:
import time
import math
import requests
import pandas as pd
headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
    'Referer':"https://www.bilibili.com/"
}
# cookies = {
#     "SESSDATA": "0ccb2e9a%2C1601886871%2C7146f*41"
# }

def Get(url):
    try:
        r = requests.get(url, headers=headers)
        return r
    except:
        return 0

#获取followings followers信息
def getFfNum(mid):
    try:
        api = "https://api.bilibili.com/x/relation/stat?vmid="+str(mid)+"&jsonp=jsonp"
        data = Get(api).json()['data']
        fi = data['following']
        fe = data['follower']
        return int(fi), int(fe)
    except:
        return -1, -1

def addDf(df, data):
    df.loc[df.shape[0],:] = data

def getFollowingsList(mid, fnum):
    if fnum <= 0:
        return []
    flist = []
    # 除20 向上取整
    page = math.ceil(fnum/20)
    for p in range(page):
        try:
            api = "https://api.bilibili.com/x/relation/followings?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
#             print(api)
            for info in Get(api).json()['data']['list']:
                flist.append(int(info['mid']))
                addDf(followdf, [mid, info['mid'] ])
        except:
            continue
    return flist

def getFollowersList(mid, fnum):
    if fnum <= 0:
        return []
    flist = []
    # 除20 向上取整
    page = math.ceil(fnum/20)
    for p in range(page):
        try:
            api = "https://api.bilibili.com/x/relation/followers?vmid="+str(mid)+"&pn="+str(p+1)+"&ps=20&order=desc&jsonp=jsonp"
#             print(api)
            for info in Get(api).json()['data']['list']:
                flist.append(int(info['mid']))
        except:
            continue

    return flist

def getUserName(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    try:
        data = Get(api).json()['data']
        return data['name']
    except:
        return 'error_'+str(mid)

def addUserDf(mid):
    api = "https://api.bilibili.com/x/space/acc/info?mid="+str(mid)+"&jsonp=jsonp"
    try:
        data = Get(api).json()['data']
        fi, fe = getFfNum(mid)
        datalist = [data['mid'], data['name'], data['sex'], data['level'], data['birthday'], data['coins'], fi, fe]
        addDf(userdf, datalist)
        
        # 后面要进行遍历，防止遍历过长，将上限改为值的3/4次幂。 例 20000-> 1684, 200000 -> 9458
#         fmax = 200000
#         if fi > fmax:
#             fi = fmax
#         if fe > fmax:
#             fe = fmax
        return math.ceil(fi**(3/4)), math.ceil(fe**(3/4)),
    except:
        return -1, -1

    
def getData(mid, gettype):
#     if mid in userdf['uid'].values:
#         return 0, []

    sleepTimes = 0
    
    fi ,fe = addUserDf(mid)
    while fi == -1:
        print('用户',str(mid),'失败')
        sleepTimes += 1
        if sleepTimes <= 3:
#             tm_mon=6, tm_mday=22, tm_hour=19, tm_min=31
            print("当前时间：",time.localtime().tm_hour," : ",time.localtime().tm_min," 休眠一次")
            time.sleep(5*60)
            fi ,fe = addUserDf(mid)
        else: return 0, []

    
    print(fi, fe)
    
    getFollowingsList(mid, fi)
    
    if gettype == 1:
        flist = getFollowersList(mid, fe)
    else : flist = []
    print('用户',str(mid),'完成')
    
    saveData()
    return 1, flist

def getDataS(li):
    for i in li:
        getData(i)

def saveData():
    userdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",  encoding='utf_8_sig')
    followdf.to_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",  encoding='utf_8_sig')
    
def roadData():
    userdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\userdf2.csv",index_col=0, dtype={'uid':int,'follower':int,'level':int,'coins':int,'following':int})
    followdf = pd.read_csv("C:\\Users\\fuwenyan\\Desktop\\b站爬虫计划\\followdf2.csv",index_col=0, dtype='int')
    return userdf, followdf

In [ ]:


# 创建用户信息数据表
# userdf = pd.DataFrame(columns=['uid', 'name', 'sex', 'level', 'birthday', 'coins', 'following', 'follower'])
userdf, followdf = roadData()
def main():
    # 从当前的关注数据中获取下一次遍历的用户数据  
    curlist = [27880221, 470156882, 282470852, 1531707, 13447830, 20964009, 297059545, 471925925, 245742750, 393673591, 18202105, 326499679, 512561912, 27113608, 188534271, 37916665, 316845538, 13187723, 457161443, 10277363, 176037767, 10119428, 486472919, 402576555, 14110780, 258150656, 431753564, 66391032, 500274206, 17024301, 435769810, 28797646, 20546476, 37663924, 378885845, 15982391, 456664753, 270105105, 450595813, 415479453, 280389001, 7792521, 407045223, 262453663, 386165680, 38351330, 19577966, 430969192, 12444306, 483203976, 387181687, 472747194, 216156027, 67214916, 203337614, 456989566, 254463269, 17546432, 3811052, 344194038, 430837803, 449825072, 382193067, 13677047, 35727555, 80717854, 927587, 20055735, 338927049, 21950148, 25782796, 407007380, 11061327, 28751839, 17075276, 6292102, 288251292, 6700597, 6302909, 18020792, 39627524, 326081112, 40966108, 23719433, 240346507, 883968, 135589565, 95304526, 16166225, 383665157, 215910135, 24127511]
#     curlist = []
#     for k in [291076700]:
#         curlist += getFollowingsList(k, 300)
    
    oknum = 0
    recurlist = []
    print(curlist)
    for c in curlist:
        isok, flist = getData(c, 1)
        oknum += isok
        recurlist.extend(flist)
        
    print("完成了",str(oknum),'个人')
    
    curlist = recurlist 
    print(curlist)
    for c in curlist:
        isok, _ = getData(c, 0)
        oknum += isok
    print("完成了",str(oknum),'个人')


main()

# getData(259755161, 0)


[719238, 125526, 37961599, 18690024, 31654362, 326246517, 119801456, 27880221, 470156882, 282470852, 1531707, 13447830, 20964009, 297059545, 471925925, 245742750, 393673591, 18202105, 326499679, 512561912, 27113608, 188534271, 37916665, 316845538, 13187723, 457161443, 10277363, 176037767, 10119428, 486472919, 402576555, 14110780, 258150656, 431753564, 66391032, 500274206, 17024301, 435769810, 28797646, 20546476, 37663924, 378885845, 15982391, 456664753, 270105105, 450595813, 415479453, 280389001, 7792521, 407045223, 262453663, 386165680, 38351330, 19577966, 430969192, 12444306, 483203976, 387181687, 472747194, 216156027, 67214916, 203337614, 456989566, 254463269, 17546432, 3811052, 344194038, 430837803, 449825072, 382193067, 13677047, 35727555, 80717854, 927587, 20055735, 338927049, 21950148, 25782796, 407007380, 11061327, 28751839, 17075276, 6292102, 288251292, 6700597, 6302909, 18020792, 39627524, 326081112, 40966108, 23719433, 240346507, 883968, 135589565, 95304526, 16166225, 383665